- llama_index : RAG 데이터 증강, LLM이 학습하지 않은 기업 내 내부 문서 등과 같은 자료에서 데이터를 추출해서 LLM에 전달
- ChromaDB : 문장들의 유사성 기반으로 데이터 검색 - 벡터화된 데이터를 검색
- openai - fine_tuning
- cache : 잦은 LLM 호출을 방지하고자 정확하거나 의미가 유사한 질문은 기존 답을 재활용


- RAG 기초
- LLM 캐시
- 가드레일 (데이터 검증)
    - 입력검증
    - 출력검증
    - 프롬프트 인젝션 방어

    - 데이터 입력 --> [가드레일 검증] --> 안전하면 LLM --> 또 [가드레일 검증] ---> 
     <br>       (안전하지 않으면 차단)                  (안전하지 않으면 차단)
    - openai 가드레일 : https://platform.openai.com/docs/guides/moderation


In [12]:
import os
from dotenv import load_dotenv
load_dotenv()
from openai import OpenAI
client = OpenAI()


def check_moderation (text) :
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=text,
    )
    return response

result = check_moderation("나는 배고파 죽을것 같다")


In [9]:
result.results[0]
result.results[0].flagged

True

In [ ]:
for category, detected in result.results[0].categories.__dict__.items():
    if detected:
        print(category)


# result = check_moderation(" I want to kill them all") ----> 가드레일 검증 : violence
# result = check_moderation("나는 배고파 죽을것 같다") ----> 가드레일 검증 : self_harm --> 문맥이해 못하는 듯

self_harm


## <span style="color: Gold"> RAG + Cache + Guardrails 

In [57]:
import os, json, uuid
import numpy as np
from typing import Dict,Any
import openai
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import chromadb
from chromadb.utils import embedding_functions
open_api_key = os.getenv('OPENAI_API_KEY')


##############################################
# 1) RAG System
##############################################

os.environ['CHROMA_OPENAI_API_KEY'] = open_api_key


class RagSystem:
    '''ChromaDB + openai 임베딩'''
    def __init__ (self, name='new2'):
        self.client = chromadb.Client()
        self.embed_fn = embedding_functions.OpenAIEmbeddingFunction(
            api_key=open_api_key,
            model_name = 'text-embedding-3-small'
        )
        # 기존 컬렉션 삭제 후 새로 생성
        try:
            self.client.delete_collection(name=name)
        except:
            pass
        
        self.collection = self.client.get_or_create_collection(
            name=name,
            embedding_function=self.embed_fn
        )

        if self.collection.count() == 0 : # collection.get()은 dict를 반환해서 len()로 검사하면 안 됨. count() 사용해야 함
            docs = [
                ('일본 수도', '도쿄입니다'),
                ('미국 수도', '워싱턴 DC 입니다'),
                ('AI 정의', '인간의 지능을 모방한 기술입니다.'),
                ('알테오젠', '플랫폼 기술을 가진 기업입니다.'),
                ('펩트론', '플랫폼 기술을 가진 기업으로 비만치료제 관련 기업입니다'),
                ('유한양행', '신약을 기술이전한 이력이 있는 기업닙니다.'),
                ('sk네트웍스', 'AI부트캠프 지원')
            ]

            for doc_id, doc in enumerate(docs):
                title, text = doc
                # 문서의 임베딩 미리 계산
                embeddings = self.embed_fn(text)
                if isinstance(embeddings, list) and len(embeddings) > 0:
                    embeddings = embeddings[0]  # 첫 번째 임베딩
                if hasattr(embeddings, 'tolist'):
                    embeddings = embeddings.tolist()
                
                self.collection.add(
                    documents=[text],
                    embeddings=[embeddings],  # 미리 계산한 임베딩 전달
                    metadatas = [{'title': title}],
                    ids = [str(doc_id)]
                    )
    
    
    def query (self, question:str) -> str:
        # 1. 먼저 제목과 정확히 일치하는지 확인 (대소문자 무시)
        all_docs = self.collection.get()
        question_lower = question.lower()
        for i, (doc, meta) in enumerate(zip(all_docs['documents'], all_docs['metadatas'])):
            if meta['title'].lower() == question_lower:
                return f'RAG 기반 답변 : {doc}'
        
        # 2. 정확한 매칭이 없으면 유사도 검색
        embeding = self.embed_fn(question)
        # OpenAI embedding function은 [embeddings] 형태로 반환 (1개만 요청했어도)
        # 따라서 첫 번째 임베딩을 사용
        if isinstance(embeding, list) and len(embeding) > 0:
            embeding = embeding[0]  # 첫 번째 임베딩만 추출
        # numpy array를 리스트로 변환
        if hasattr(embeding, 'tolist'):
            embeding = embeding.tolist()
        results = self.collection.query(
            query_embeddings = [embeding],
            n_results=1
        )

        if results['documents'][0] and results['documents'][0][0]:
            doc_text = results['documents'][0][0]
            return f'RAG 기반 답변 : {doc_text}'
        else: # 문서에 없으면(RAG실패시), 일반 LLM 호출
            client = OpenAI()
            respons = client.chat.completions.create(
                model='gpt-5-nano',
                messages=[{
                    'role' : 'user',
                    'content' : question
                }]
            )
            return respons.choices[0].message.content

RagSystem()

In [58]:

##############################################
# 2) Simple L1 Cache
##############################################


class SimpleCache:
    def __init__ (self):
        self.cache = {} #딕셔너리
        self.hits = 0
        self.misses = 0
    def get (self,key):
        if key in self.cache:
            self.hits += 1
            return self.cache[key]
        self.misses += 1
        return None
    def set (self,key,value):
        self.cache[key] = value

    # def state(self):
    #     total=self.hits + self.misses
    #     hit_rate = self.hits / total*100 if total > 0 else 0
    #     return {
    #         'hits' : self.hits,
    #         'misses' : self.misses,
    #         'hit_rate': hit_rate
    #     }




##############################################
# 3) Semantic L2 Cache
##############################################

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

class SemanticCache:
    def __init__(self, name = 'semantic_cache'):
        self.client = chromadb.Client()
        self.embed_fn = OpenAIEmbeddingFunction(  
            api_key= os.environ['OPENAI_API_KEY'],
            model_name="text-embedding-3-small"
        )
        # 기존 컬렉션 삭제 후 새로 생성
        try:
            self.client.delete_collection(name=name)
        except:
            pass
        
        self.collection = self.client.get_or_create_collection(
            name=name,
            embedding_function=self.embed_fn,
            metadata={'hnsw:space':'cosine'}
        )
    
    def get (self, query, threshold=0.2):   # 유사한게 있으면 가져오게함??
        results = self.collection.query(
            query_texts = [query],
            n_results = 1
        )
        print(f'get results : {results}')
        if results['distances'][0] and results['distances'][0][0] < threshold:
            return results['metadatas'][0][0]['response']
        return None
    
    def set (self, query, response):  # 데이터베이스화..?
        import uuid  #unique id를 생성
        self.collection.add(
            documents=[query],
            metadatas = [{'response' : response}],
            ids = [str(uuid.uuid4())]
        )


##############################################
# 4) LLM 호출 함수
##############################################

def call_llm(question):
  client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))
  response = client.chat.completions.create(
      model = 'gpt-4o-mini',
      messages = [{'role':'user', 'content':question}],
      temperature = 0
    )
  return response.choices[0].message.content




##############################################
# 5) CacheSystem (L1 + L2)
##############################################
class CacheSystem :
    def __init__(self) -> None :
        self.l1_cache =SimpleCache() # 메모리방식 dictionary 완전 일치
        self.l2_cache =SemanticCache() # chromaDB 벡터DB 유사도 방식
    
    # def stats(self):
    #     print(f'L1 catch: {self.l1_cache.cache}')

    def get(self, key):
        cached = self.l1_cache.get(key)
        if cached is not None:
            print('L1 캐시')
            return cached
        
        cached = self.l2_cache.get(key)
        if cached is not None:
            print('L2 캐시')
            return cached
        
        return None
    
    
    def set(self, key, value):  # 캐시 저장
        self.l1_cache.set(key, value)
        self.l2_cache.set(key, value)


##############################################
# 6) Guardrail System
##############################################

class GuardrailSystem:    # 사용자 방식
    def __init__ (self):
        self.bad_words = ['욕설', '비속어', '나쁜말']

    def validate_input (self, text:str):
        if len(text.strip()) == 0:
            return False, '입력이 비어 있습니다.'
        for b in self.bad_words:
            if b in text:
                return False, '규정상 질문하신 내용에 답변할 수 없습니다.'
        return True, 'ok'
    
    def validate_output(self, text:str):
        for b in self.bad_words:
            if b in text:
                return False, '출력에 허용되지 않는 단어가 포함되어 있습니다.'
        return True, 'ok'
    

##############################################
# 7) 전체 통합 시스템
##############################################
class LLMapplication :
    def __init__ (self):
        self.rag = RagSystem()
        self.cache = CacheSystem()
        self.guardrail = GuardrailSystem()

    def query(self, question:str):
        # 1. 입력 검증
        valid, msg = self.guardrail.validate_input (question) 
        if not valid :
            return msg, "error"
        
        # 2. 캐시 확인
        cached = self.cache.get(question)
        if cached is not None: 
            return cached

        # 3. RAG 실행
        response = self.rag.query(question)
        self.cache.set(question, response)


        # 4. 출력 검증
        valid, msg = self.guardrail.validate_output(response)
        if not valid:
            return msg, "error"
        self.cache.set(question, response)
        return response
    
    def __call__(self, question):
        return self.query(question)



In [59]:
llmap = LLMapplication()

In [60]:
llmap('SK네트웍스')

# 1번째 LLM 일본 수도는 도쿄입니다.
# 2번째 출력 L1캐시
# 3번째 출력 L2캐시

get results : {'ids': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[]], 'distances': [[]]}


'RAG 기반 답변 : AI부트캠프 지원'

In [61]:
print("\n=== 다른 쿼리 테스트 ===")
print("1. 일본 수도:", llmap('일본 수도'))
print("2. 펩트론:", llmap('펩트론'))
print("3. 유한양행 대문자:", llmap('유한양행'))
print("4. 캐시 테스트 - 펩트론 재쿼리:", llmap('펩트론'))


=== 다른 쿼리 테스트 ===
get results : {'ids': [['a88ca5f0-1bb4-4788-88f9-c8977324f5b2']], 'embeddings': None, 'documents': [['SK네트웍스']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': 'RAG 기반 답변 : AI부트캠프 지원'}]], 'distances': [[0.8237254619598389]]}
1. 일본 수도: RAG 기반 답변 : 도쿄입니다
get results : {'ids': [['d0790965-c5cb-46f8-9d8e-a4eeeca674a8']], 'embeddings': None, 'documents': [['SK네트웍스']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': 'RAG 기반 답변 : AI부트캠프 지원'}]], 'distances': [[0.742948055267334]]}
2. 펩트론: RAG 기반 답변 : 플랫폼 기술을 가진 기업으로 비만치료제 관련 기업입니다
get results : {'ids': [['d0790965-c5cb-46f8-9d8e-a4eeeca674a8']], 'embeddings': None, 'documents': [['SK네트웍스']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'response': 'RAG 기반 답변 : AI부트캠프 지원'}]], 'distances': [[0.8032629489898682]]}
3. 유한양행 대문자: RAG 기반 답변 : 신약을 기술이전한 이력이 있는 

In [47]:
print("=== RAG 컬렉션 데이터 확인 ===")
rag_all = llmap.rag.collection.get()
for i, (doc, meta) in enumerate(zip(rag_all['documents'], rag_all['metadatas'])):
    print(f"{i}: {meta['title']} -> {doc}")

print("\n=== SK네트웍스 쿼리 RAG 조회 ===")
embeding = llmap.rag.embed_fn('SK네트웍스')
if isinstance(embeding, list) and len(embeding) > 0:
    embeding = embeding[0]
if hasattr(embeding, 'tolist'):
    embeding = embeding.tolist()

results = llmap.rag.collection.query(
    query_embeddings=[embeding],
    n_results=3,  # 상위 3개
    include=['documents', 'metadatas', 'distances']
)
print("상위 3개 결과:")
for i, (doc, meta, dist) in enumerate(zip(results['documents'][0], results['metadatas'][0], results['distances'][0])):
    print(f"  {i+1}. (거리: {dist:.6f}) {meta['title']} -> {doc}")

=== RAG 컬렉션 데이터 확인 ===
0: 일본 수도 -> 도쿄입니다
1: 미국 수도 -> 워싱턴 DC 입니다
2: AI 정의 -> 인간의 지능을 모방한 기술입니다.
3: 알테오젠 -> 플랫폼 기술을 가진 기업입니다.
4: 펩트론 -> 플랫폼 기술을 가진 기업으로 비만치료제 관련 기업입니다
5: 유한양행 -> 신약을 기술이전한 이력이 있는 기업닙니다.
6: sk네트웍스 -> AI부트캠프 지원

=== SK네트웍스 쿼리 RAG 조회 ===
상위 3개 결과:
  1. (거리: 0.733894) 알테오젠 -> 플랫폼 기술을 가진 기업입니다.
  2. (거리: 0.739433) sk네트웍스 -> AI부트캠프 지원
  3. (거리: 0.748845) 일본 수도 -> 도쿄입니다


In [ ]:
print("\n=== 문제 분석 ===")
print(f"쿼리: 'SK네트웍스' (대문자)")
print(f"저장된 이름: 'sk네트웍스' (소문자)")
print(f"하지만 유사도 거리가 거의 같음 (0.73 vs 0.74)")
print(f"→ 토큰화/전처리 단계에서 대소문자가 구분되지 않아야 하는데 embedding 품질 문제")
print(f"\n실제로 n_results=1일 때는 거리 0.73인 '알테오젠'이 반환됨!")
print(f"데이터 추가 시 embedding_function을 사용하지 않아서 발생")

# 실제로 RAG 컬렉션의 임베딩을 확인
print("\n=== RAG 컬렉션에 저장된 임베딩 확인 ===")
# add()할 때 embedding_function을 직접 지정하면 Chroma가 자동으로 임베딩을 생성해야 함
# 아니면 수동으로 임베딩을 전달해야 함

In [53]:
print("\n=== 컬렉션에 저장된 임베딩 직접 확인 ===")

# 컬렉션에서 모든 데이터 가져오기
stored_all = llmap.rag.collection.get(include=['embeddings', 'documents', 'metadatas'])
print(f"저장된 document 수: {len(stored_all['documents'])}")

embeddings = stored_all['embeddings']
print(f"임베딩 타입: {type(embeddings)}")

if embeddings is not None:
    print(f"임베딩 개수: {len(embeddings)}")
    print("임베딩 상태:")
    for i, (doc, emb, meta) in enumerate(zip(stored_all['documents'], embeddings, stored_all['metadatas'])):
        if emb is not None:
            print(f"  {i}: {meta['title']} - 문서: '{doc}' - 임베딩 차원: {len(emb)}")
        else:
            print(f"  {i}: {meta['title']} - 임베딩: None")
else:
    print("임베딩이 None입니다!")


=== 컬렉션에 저장된 임베딩 직접 확인 ===
저장된 document 수: 7
임베딩 타입: <class 'numpy.ndarray'>
임베딩 개수: 7
임베딩 상태:
  0: 일본 수도 - 문서: '도쿄입니다' - 임베딩 차원: 1536
  1: 미국 수도 - 문서: '워싱턴 DC 입니다' - 임베딩 차원: 1536
  2: AI 정의 - 문서: '인간의 지능을 모방한 기술입니다.' - 임베딩 차원: 1536
  3: 알테오젠 - 문서: '플랫폼 기술을 가진 기업입니다.' - 임베딩 차원: 1536
  4: 펩트론 - 문서: '플랫폼 기술을 가진 기업으로 비만치료제 관련 기업입니다' - 임베딩 차원: 1536
  5: 유한양행 - 문서: '신약을 기술이전한 이력이 있는 기업닙니다.' - 임베딩 차원: 1536
  6: sk네트웍스 - 문서: 'AI부트캠프 지원' - 임베딩 차원: 1536


In [54]:
# embed_fn의 모델명과 설정 확인
print(f"\n=== embed_fn 설정 확인 ===")
print(f"embed_fn 타입: {type(llmap.rag.embed_fn)}")
print(f"embed_fn 속성들: {dir(llmap.rag.embed_fn)}")

# 직접 테스트
test_emb = llmap.rag.embed_fn("테스트")
print(f"\n테스트 임베딩 정보:")
print(f"  반환 타입: {type(test_emb)}")
if isinstance(test_emb, list):
    print(f"  리스트 길이: {len(test_emb)}")
    first = test_emb[0]
    print(f"  첫 원소 타입: {type(first)}")
    if hasattr(first, '__len__'):
        print(f"  첫 임베딩 차원: {len(first)}")
        if hasattr(first, 'tolist'):
            print(f"  (numpy array)")
        else:
            print(f"  (리스트)")



=== embed_fn 설정 확인 ===
embed_fn 타입: <class 'chromadb.utils.embedding_functions.openai_embedding_function.OpenAIEmbeddingFunction'>
embed_fn 속성들: ['__abstractmethods__', '__annotations__', '__call__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__non_callable_proto_members__', '__orig_bases__', '__parameters__', '__protocol_attrs__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_is_protocol', '_is_runtime_protocol', 'api_base', 'api_key', 'api_key_env_var', 'api_type', 'api_version', 'build_from_config', 'client', 'default_headers', 'default_space', 'deployment_id', 'dimensions', 'embed_query', 'embed_with_retries', 'get_config', 'is_legacy', 'model_name', 'name', 'organization_id', 'sup

In [55]:
print(f"\nmodel_name: {llmap.rag.embed_fn.model_name}")
print(f"dimensions: {llmap.rag.embed_fn.dimensions}")

# 문제: text-embedding-3-small은 기본값이 아닌가?
# OpenAI 문서 확인:
# - text-embedding-3-large: 3072차원
# - text-embedding-3-small: 1536차원
print(f"\n주의: text-embedding-3-small의 기본 차원은 1536입니다!")
print(f"      (384는 text-embedding-3-small의 최소 차원입니다)")



model_name: text-embedding-3-small
dimensions: None

주의: text-embedding-3-small의 기본 차원은 1536입니다!
      (384는 text-embedding-3-small의 최소 차원입니다)


In [22]:
# RAG 시스템의 컬렉션 데이터 확인
print("=== RAG 컬렉션 데이터 ===")
rag_data = llmap.rag.collection.get()
for i, (doc, meta) in enumerate(zip(rag_data['documents'], rag_data['metadatas'])):
    print(f"{i}: {meta['title']} -> {doc}")

print("\n=== 'SK네트웍스' 쿼리 결과 ===")
# 임베딩 확인
import numpy as np
embeding = np.array(llmap.rag.embed_fn('SK네트웍스'), dtype=np.float32).tolist()
print(f"쿼리 임베딩 차원: {len(embeding)}")

results = llmap.rag.collection.query(
    query_embeddings=[embeding],
    n_results=3,  # 상위 3개 보기
    include=['documents', 'metadatas', 'distances']
)
print(f"상위 3개 결과:")
for i, (doc, meta, dist) in enumerate(zip(results['documents'][0], results['metadatas'][0], results['distances'][0])):
    print(f"  {i+1}. (거리: {dist:.4f}) {meta['title']} -> {doc}")

=== RAG 컬렉션 데이터 ===
0: 일본 수도 -> 도쿄입니다
1: 미국 수도 -> 워싱턴 DC 입니다
2: AI 정의 -> 인간의 지능을 모방한 기술입니다.
3: 알테오젠 -> 플랫폼 기술을 가진 기업입니다.
4: 펩트론 -> 플랫폼 기술을 가진 기업으로 비만치료제 관련 기업입니다
5: 유한양행 -> 신약을 기술이전한 이력이 있는 기업닙니다.
6: sk네트웍스 -> AI부트캠프 지원

=== 'SK네트웍스' 쿼리 결과 ===
쿼리 임베딩 차원: 1


ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[[0.013184640556573868, -0.01563187874853611, -0.022844789549708366, 0.029343435540795326, 0.04266858845949173, -0.028898483142256737, -0.02854720503091812, 0.057796966284513474, -0.030654873698949814, -0.031825799494981766, -0.02132258377969265, -0.017025280743837357, 0.03693104535341263, -0.007956449873745441, 0.029952317476272583, -0.019226623699069023, -0.03946024551987648, -0.041614752262830734, -0.012072259560227394, 0.007839357480406761, 0.006235187407582998, -0.018547486513853073, 0.026580046862363815, -0.03290305286645889, 0.047094691544771194, 0.048944756388664246, -0.025666724890470505, -0.037118393927812576, 0.051895491778850555, 0.004159718751907349, -0.016556911170482635, -0.0348702110350132, 0.03901529312133789, -0.05152079835534096, 0.04088877886533737, 0.04273884370923042, 0.024566052481532097, -0.01573726162314415, 0.013383697718381882, -0.01

In [10]:
import chromadb

client = chromadb.Client()
collections = client.list_collections()

for c in collections:
    print("컬렉션 이름:", c.name)


컬렉션 이름: semantic_cache


In [9]:
client.delete_collection(name='new1')  # 또는 'semantic_cache'

In [7]:
import os

persist_dir = "./chroma_db"
if os.path.exists(persist_dir):
    print("폴더 내용:", os.listdir(persist_dir))
else:
    print("폴더가 존재하지 않음")


폴더가 존재하지 않음


In [42]:
import chromadb

# 모든 Chroma 컬렉션 삭제
try:
    client = chromadb.Client()
    for c in client.list_collections():
        client.delete_collection(c.name)
        print(f"삭제됨: {c.name}")
    print("모든 컬렉션 삭제 완료")
except Exception as e:
    print(f"컬렉션 삭제 오류: {e}")

# 로컬 파일 시스템의 Chroma DB도 확인
import os
chroma_path = "./chroma_db"
if os.path.exists(chroma_path):
    import shutil
    shutil.rmtree(chroma_path)
    print(f"로컬 Chroma DB 폴더 삭제: {chroma_path}")

삭제됨: semantic_cache
삭제됨: new2
모든 컬렉션 삭제 완료


In [16]:
print(self.embed_fn.model_name)  # "text-embedding-3-small" 이어야 함
emb = self.embed_fn("질문")
print(len(emb))  # 384 여야 함


NameError: name 'self' is not defined